### 1. Assignment Topic:

In this project, you will build a regression model using the Keras library to model the same data about concrete compressive strength that we used in labs 3.p In this course project, you will build a regression model using the deep learning Keras library, and then you will experiment with increasing the number of training epochs and changing number of hidden layers and you will see how changing these parameters impacts the performance of the model.

## Increasing the Number of Training Epochs - 2

In [21]:
import pandas as pd
import numpy as np

In [22]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [23]:
concrete_data.shape

(1030, 9)

In [24]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [25]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [26]:
concrete_data_columns = concrete_data.columns

target = concrete_data["Strength"]
predictors = concrete_data.drop(["Strength", "Age"], axis=1)

In [27]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [28]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [29]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [30]:
n_cols = predictors_norm.shape[1]  # number of predictors
n_cols

7

In [31]:
import warnings
warnings.filterwarnings("ignore")
import keras

In [32]:
from keras.models import Sequential
from keras.layers import Dense

In [33]:
# define regression model
def regression_model():
    # 1-Create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # 2-Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [36]:
model = regression_model()

In [37]:
epochs = 120
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/120
23/23 [==============================] - 0s 1ms/step - loss: 656.5568
Epoch 2/120
23/23 [==============================] - 0s 1ms/step - loss: 230.7215
Epoch 3/120
23/23 [==============================] - 0s 1ms/step - loss: 203.2370
Epoch 4/120
23/23 [==============================] - 0s 1ms/step - loss: 185.4575
Epoch 5/120
23/23 [==============================] - 0s 1ms/step - loss: 177.7821
Epoch 6/120
23/23 [==============================] - 0s 1ms/step - loss: 175.7510
Epoch 7/120
23/23 [==============================] - 0s 1ms/step - loss: 165.8702
Epoch 8/120
23/23 [==============================] - 0s 2ms/step - loss: 171.5458
Epoch 9/120
23/23 [==============================] - 0s 1ms/step - loss: 163.8970
Epoch 10/120
23/23 [==============================] - 0s 1ms/step - loss: 162.8696
Epoch 11/120
23/23 [==============================] - 0s 1ms/step - loss: 155.3817
Epoch 12/120
23/23 [==============================] - 0s 1ms/step - loss: 166.2188
Epoch 13/120


23/23 [==============================] - 0s 1ms/step - loss: 140.2031
Epoch 100/120
23/23 [==============================] - 0s 1ms/step - loss: 142.7977
Epoch 101/120
23/23 [==============================] - 0s 1ms/step - loss: 151.0708
Epoch 102/120
23/23 [==============================] - 0s 1ms/step - loss: 157.7865
Epoch 103/120
23/23 [==============================] - 0s 1ms/step - loss: 141.5477
Epoch 104/120
23/23 [==============================] - 0s 1ms/step - loss: 131.5896
Epoch 105/120
23/23 [==============================] - 0s 1ms/step - loss: 143.9551
Epoch 106/120
23/23 [==============================] - 0s 1ms/step - loss: 137.6365
Epoch 107/120
23/23 [==============================] - 0s 1ms/step - loss: 163.9671
Epoch 108/120
23/23 [==============================] - 0s 1ms/step - loss: 168.4381
Epoch 109/120
23/23 [==============================] - 0s 1ms/step - loss: 179.5814
Epoch 110/120
23/23 [==============================] - 0s 1ms/step - loss: 135.0195
Epoch 

In [39]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 1ms/step - loss: 162.4939


162.493896484375

In [40]:
from sklearn.metrics import mean_squared_error

In [41]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(f"Mean: {mean}\nStandard Deviation: {standard_deviation}")

Mean: 162.49391533142077
Standard Deviation: 0.0
